# Python code for Single-Cell pairs.txt files 
This step will generate different pairs txt fles for different resolutions of the Single-Cell data

### Import packages


In [1]:
import numpy as np
from PIL import Image
import os
import csv
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def find_replace(text_old,text_new,file_full_path):
    # Read in the file
    with open(file_full_path, 'r') as file:
        filedata = file.read()

    # Replace the target string
    filedata = filedata.replace(text_old, text_new)

    # Write the file out again
    with open(file_full_path, 'w') as file:
        file.write(filedata)

### Import data

In [3]:
# Set the path to the directory containing pairs
input_directory = "/media/msbb/ssd2/Yasmine_copy/Sci_HiC_pairs/GSE208045_sci_ra_pairs/"
# Create a new directory called "pairs"
save_directory = "/media/msbb/ssd2/Yasmine_copy/Sci_HiC_pairs/Thp1_monocyte_pairs"
os.makedirs(save_directory, exist_ok=True)

### Loop through files to make new Dataframes

In [4]:
# Define the function to find and replace text in a file
def find_replace(text_old, text_new, file_full_path):
    # Read in the file
    with open(file_full_path, 'r') as file:
        filedata = file.read()

    # Replace the target strings
    filedata = filedata.replace(text_old, text_new)

    # Write the file out again
    with open(file_full_path, 'w') as file:
        file.write(filedata)

# For the Thp1 macrophages infected with Mtb
input_names = os.listdir(input_directory)
input_paths = [os.path.join(input_directory, input_name) for input_name in input_names if input_name.endswith(".txt")]

# Step 1: Check for Errors and Rename Columns
for input_name, input_path in zip(input_names, input_paths):
    # Read chrom file and add it to DataFrame
    COLUMN_NAMES = ['readID', 'chr1', 'position1', 'chr2', 'position2', 'strand1', 'strand2']
    data = pd.read_csv(input_path, sep='\t', names=COLUMN_NAMES, skiprows=2)
    
    # Select and rename columns
    data = data[['chr1', 'position1', 'chr2', 'position2']]        
    data.rename(columns={'chr1': '#chr_A', 'position1': 'pos_A', 'chr2': 'chrB', 'position2': 'pos_B'}, inplace=True)
    
    # Save the modified DataFrame as a new txt file in the pairs directory
    output_filename = os.path.join(save_directory, input_name)
    data.to_csv(output_filename, sep='\t', index=False)
    
    # Find and replace "chromosome X" and "chromosome Y" in the original input file
    find_replace('chrX', 'chr23', input_path)
    find_replace('chrY', 'chr24', input_path)

# Step 2: Check Data Reading
print("Input Paths:")
print(input_paths)

for input_name, input_path in zip(input_names, input_paths):
    print("Processing:", input_name)
    print("Input Path:", input_path)
    # Read chrom file and add it to DataFrame
    COLUMN_NAMES = ['readID', 'chr1', 'position1', 'chr2', 'position2', 'strand1', 'strand2']
    data = pd.read_csv(input_path, sep='\t', names=COLUMN_NAMES, skiprows=2)
    print("Data:")
    print(data.head())  # Print the first few rows of the DataFrame

# Step 3: Check DataFrame Operations
for input_name, input_path in zip(input_names, input_paths):
    # Read chrom file and add it to DataFrame
    COLUMN_NAMES = ['readID', 'chr1', 'position1', 'chr2', 'position2', 'strand1', 'strand2']
    data = pd.read_csv(input_path, sep='\t', names=COLUMN_NAMES, skiprows=2)
    
    # Select and rename columns
    data = data[['chr1', 'position1', 'chr2', 'position2']]        
    data.rename(columns={'chr1': '#chr_A', 'position1': 'pos_A', 'chr2': 'chrB', 'position2': 'pos_B'}, inplace=True)
    print("Data after renaming:")


FileNotFoundError: [Errno 2] No such file or directory: '/media/msbb/ssd2/Yasmine_copy/Sci_HiC_pairs/GSE208045_sci_ra_pairs/'

## Creating HiC from Single-Cell pairs files

In order to retrieve the Single-Cell HiC data, we need to run the Juicertools on all the pairs files from the Single-Cell

In [55]:
# Directory for import data
pairs_directory= '/media/msbb/ssd2/Yasmine_copy/Sci_pairs/GSE208045_sci_thpp_pairs/sci_thpp_pairs_txt/'

# Create a new directory called "Single_Cell_HiC"
Hic_directory = '/media/msbb/ssd2/Yasmine_copy/Single_Cell_HiC/Sci_HiC/Resolution_5e5/Macrophage-Mtb_Sci_HiC/'
os.makedirs(Hic_directory, exist_ok=True)

# List all files with the .txt extension in the pairs directory
pairs_files = [pairs_file for pairs_file in os.listdir(pairs_directory) if pairs_file.endswith(".txt")]

In [56]:
# Loop through each pairs file and generate the corresponding .hic file
for pairs_file in pairs_files:
    # Paths for input and output files
    input_file_path = os.path.join(pairs_directory, pairs_file)
    output_file_path = os.path.join(Hic_directory, os.path.splitext(pairs_file)[0] + ".hic")
    
    # Run the juicer_tools command for each file
    command = f'java -jar juicer_tools_1.22.01.jar pre "{input_file_path}" "{output_file_path}" hg19 -r 500000'
    os.system(command)

        

In [22]:
# Set the path to the directory containing hic files
hic_directory = "/media/msbb/ssd2/Yasmine_copy/Single_Cell_HiC/Sci_HiC/Resolution_10e3/Macrophage_Sci_HiC/"

# Create a new directory called "pairs"
pairs_directory = "/media/msbb/ssd2/Yasmine_copy/Single_Cell_HiC/Sci_Pairs_resolutions_Macrophages"

os.makedirs(pairs_directory, exist_ok=True)

hic_files = os.listdir(hic_directory)
hic_files = [hic_file for hic_file in hic_files if hic_file.endswith(".hic")]
hic_files_names = [hic_file[:-4] for hic_file in hic_files]
hic_files_paths = [f"{hic_directory}/{hic_file}" for hic_file in hic_files]
discretisation_steps = [10e3, 5e4, 10e4, 5e5, 10e5]

for step in discretisation_steps:
    step = int(step)
    for hic_file_name, hic_file_path in zip(hic_files_names, hic_files_paths):
        # Loop through each hic file in the directory

        # Loop through each chromosome, including X and Y
        all_pair_df = pd.DataFrame(columns=['#chrA', 'pos_A', 'chrB', 'pos_B'])
        if not os.path.isdir(f'{pairs_directory}/{step}/{hic_file_name}'):
            os.makedirs(f'{pairs_directory}/{step}/{hic_file_name}')

        for chr in range(1, 25):
            if chr == 23:
                chr_name = "X"
            elif chr == 24:
                chr_name = "Y"
            else:
                chr_name = str(chr)

            # Construct the new file name without the hic extension and with the chromosome name
            new_file_name = f'{pairs_directory}/{step}/{hic_file_name}/{hic_file_name}_{chr_name}.pairs.txt'

            # Run juicer command using double quotes to wrap the entire command
            os.system(
                f'java -jar juicer_tools_1.22.01.jar dump observed NONE {hic_file_path} chr{chr_name} chr{chr_name} BP {step} {new_file_name}')
            print(f"Processed: {hic_file_name} chromosome {chr_name}")

            # read chrom file and add it to DataFrame
            COLUMN_NAMES = ['pos_A', 'pos_B']
            data = pd.read_csv(new_file_name, sep='\t', names=COLUMN_NAMES)

            # add new columns encoding chrom numbers
            data['#chrA'] = ['chr' + str(chr_name)] * len(data)
            data['chrB'] = ['chr' + str(chr_name)] * len(data)

            # swap the columns to have CT code readable format
            data = data.reindex(columns=['#chrA', 'pos_A', 'chrB', 'pos_B'])

            # add one more chrom to all croms data frame
            all_pair_df = pd.concat([all_pair_df, data], ignore_index=True, sort=False)

        # save df as txt
        all_pair_df.to_csv(f'{new_file_name[:-12]}_all.pairs.txt', sep='\t', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/media/msbb/ssd2/Yasmine_copy/Single_Cell_HiC/Sci_HiC/Resolution_10e3/Macrophage_Sci_HiC/'

## Process to fasten the generation of the txt files
The following code can be used to fasten the time it takes to generate the files. This code uses multiple cores of the computer system.

In [3]:
def process_the_cell(hic_file_base_name, 
                     hic_file_path, 
                     step, 
                     pairs_output_directory, 
                     ):
    
    all_pair_df = pd.DataFrame(columns=['#chrA', 'pos_A', 'chrB', 'pos_B', 'pairs_of_contacts'])

    
    for chr_num in range(1, 25):
        if chr_num == 23:
            chr_name = "X"
        elif chr_num == 24:
            chr_name = "Y"
        else:
            chr_name = str(chr_num)

        # Construct the new file name without the .hic extension and with the chromosome name
        new_file_name = f'{hic_file_base_name}_{chr_name}.pairs.txt'

        # Run juicer command using double quotes to wrap the entire command
        print(step)
        os.system(
            f'java -jar juicer_tools_1.22.01.jar dump observed NONE {hic_file_path} chr{chr_name} chr{chr_name} BP {int(float(step))} {os.path.join(pairs_output_directory, new_file_name)}')
        print(f"Processed: {hic_file_base_name} chromosome {chr_name} at {step} bp")

        # Read the pairs file and add it to DataFrame
        COLUMN_NAMES = ['pos_A', 'pos_B', 'pairs_of_contacts']
        data = pd.read_csv(os.path.join(pairs_output_directory, new_file_name), sep='\t', names=COLUMN_NAMES)

        # Add new columns encoding chromosome numbers
        data['#chrA'] = ['chr' + str(chr_name)] * len(data)
        data['chrB'] = ['chr' + str(chr_name)] * len(data)

        # Swap the columns to have CT code readable format
        data = data.reindex(columns=['#chrA', 'pos_A', 'chrB', 'pos_B','pairs_of_contacts'])

        # add one more chrom to all croms data frame
        all_pair_df = pd.concat([all_pair_df, data], ignore_index=True, sort=False)

    # save df as txt
    all_pair_df.to_csv(f'{pairs_output_directory}/{new_file_name[:-12]}_all.pairs.txt', sep='\t', index=False)


In [4]:
def process_hic_files(hic_directory, pairs_directory, step):
    hic_files = os.listdir(hic_directory)
    hic_files = [hic_file for hic_file in hic_files if hic_file.endswith(".hic")]

    for hic_file_name in hic_files:
        hic_file_path = os.path.join(hic_directory, hic_file_name)
        hic_file_base_name = hic_file_name[:-4]
        
        
        pairs_output_directory = os.path.join(pairs_directory, hic_file_base_name)
        # print(hic_output_directory)
        if not os.path.exists(pairs_output_directory):
            os.makedirs(pairs_output_directory, exist_ok=True)

            process_the_cell(hic_file_base_name,
                         hic_file_path,
                         step,
                         pairs_output_directory,
                         )          

### Parallel execution

In [33]:
from multiprocessing import Pool

cell_types = ['Macrophage','Macrophage-Mtb','Monocyte']

#Running the process 

discretisation_steps = ["10e3", "5e4", "10e4", "5e5", "10e5"]


hic_directory_list = []
pairs_directory_list = []
discretisation_steps_list = []

for step in discretisation_steps:
    for cell_type in cell_types:

        hic_directory = f"/media/msbb/ssd2/Yasmine_copy/Single_Cell_HiC/Sci_HiC/{step}/{cell_type}_Sci_HiC/"
        pairs_directory = f"/media/msbb/ssd2/Yasmine_copy/Sci_pairs_resolutions/Sci_Pairs_Parallel/{step}/{cell_type}_Sci_pairs"

        hic_directory_list.append(hic_directory)
        pairs_directory_list.append(pairs_directory)
        discretisation_steps_list.append(step)



with Pool(15) as p:
    p.starmap(process_hic_files, zip(hic_directory_list,pairs_directory_list,discretisation_steps_list))